In [1]:
# rSLDS actual data 

In [ ]:
# RUN IN COLAB ONLY: install required modules: ssm, openpyxl
!pip -q install git+https://github.com/lindermanlab/ssm.git #egg=ssm
!pip -q install openpyxl

In [ ]:
# RUN IN COLAB ONLYL: force-fetch latest files from GitHub (overwrite local copies)
!curl -L -H "Cache-Control: no-cache" -o gridsearch.py https://raw.githubusercontent.com/chrismader1/SLDS/main/gridsearch.py
!curl -L -H "Cache-Control: no-cache" -o rSLDS.py https://raw.githubusercontent.com/chrismader1/SLDS/main/rSLDS.py

In [ ]:
# RUN IN COLAB ONLYL: create a subfolder /Data for manual transfer 
import os
os.makedirs("Data", exist_ok=True)
# Copy-paste the following files into the Data subfolder:
# > bbg_data.xlsx
# > F-F_Momentum_Factor_daily.csv
# > F-F_Research_Data_5_Factors_2x3_daily.csv
# > F-F_Research_Data_Factors_daily.csv

In [ ]:
# RUN IN COLAB ONLY: prevent timeouts
%%javascript
// Ping the backend + re-click the connect button every 60s
(function keepAlive(){
  const clickConnect = () => {
    const btn = document.querySelector('colab-connect-button') || document.querySelector('#connect');
    if (btn) btn.click();
    console.log('keepAlive: ping');
  };
  setInterval(clickConnect, 60000);
})();


In [3]:
import numpy as np
import pandas as pd
# sys.path.insert(0, "/Users/chrismader/Python/SLDS")
from gridsearch import pipeline_actual

In [4]:
# --------------------------------------------------------------------------------------
# CONFIG
# --------------------------------------------------------------------------------------

CONFIG = {
    
    # Core defaults
    "n_jobs": -1,  # multi-threading
    "dt": 1.0 / 252.0,
    "n_iters": 50,
    "h_z": 3.0,  # CUSUM parameter
    
    # Batch windows
    "batch_grid": [
        {"train_window": 756, "overlap_window": 5},
        # {"train_window": 256, "overlap_window": 63},
        # {"train_window": 504, "overlap_window": 63},
        # {"train_window": 756, "overlap_window": 63},
        # {"train_window": 1260, "overlap_window": 63},
    ],

    # Number of regimes
    "K_grid": [2, 3, 4],
    
    # Unrestricted models: 
    "unrestricted_models": [
       {"label": "[y]",         "channels": ["y"],                "dim_latent": [1]},
        {"label": "[y,h]",       "channels": ["y","h"],            "dim_latent": [2]},
        {"label": "[g,v]",       "channels": ["g","v"],            "dim_latent": [2]},
        {"label": "[g,v,h]",     "channels": ["g","v","h"],        "dim_latent": [2,3]},
        {"label": "[y,g,v,h]",   "channels": ["y","g","v","h"],    "dim_latent": [3,4]},
    ],

    # Restricted models: 
    "restricted_models": [
        {"label": "fund1",        "channels": ["y"],                 "dim_latent": [2],    "C_type": "fund1"},
        {"label": "fund1_vix",    "channels": ["y","h"],             "dim_latent": [3],    "C_type": "fund1_vix"},
        {"label": "fund2",        "channels": ["y","g"],             "dim_latent": [2],    "C_type": "fund2"},
        {"label": "fund2_vix",    "channels": ["y","g","h"],         "dim_latent": [3],    "C_type": "fund2_vix"},
        {"label": "fund3",        "channels": ["y","v","g"],         "dim_latent": [2],    "C_type": "fund3"},
        {"label": "fund3_vix",    "channels": ["y","v","g","h"],     "dim_latent": [3],    "C_type": "fund3_vix"},   

        {"label": "factor1",      "channels": ["y"],                 "dim_latent": [2],    "C_type": "factor1"},
        {"label": "factor1_vix",  "channels": ["y","h"],             "dim_latent": [3],    "C_type": "factor1_vix"},

        {"label": "factor2_ff3",   "channels": ["y","mkt","smb","hml"],                   "dim_latent": [3], "C_type": "factor2"},
        {"label": "factor2_ff3mom","channels": ["y","mkt","smb","hml","mom"],             "dim_latent": [4], "C_type": "factor2"},
        {"label": "factor2_ff5",   "channels": ["y","mkt","smb","hml","rmw","cma"],       "dim_latent": [5], "C_type": "factor2"},
        {"label": "factor2_ff5mom","channels": ["y","mkt","smb","hml","rmw","cma","mom"], "dim_latent": [6], "C_type": "factor2"},
    ],

    # Model selection
    "run_unrestricted": True,
    "run_restricted": True,

    # Output
    "display": False,
    "verbose": False,    
}


In [5]:
# --------------------------------------------------------------------------------------
# Execute
# --------------------------------------------------------------------------------------

def main():

    # filename = "/Users/chrismader/Python/SLDS/Data/bbg_data.xlsx"
    filename = "Data/bbg_data.xlsx"
    
    # df = pd.read_excel(filename, sheet_name="SPX_PX", skiprows=4, index_col=0)
    # df = df.iloc[3:, :]
    # securities = df.columns.to_list()
    
    securities = ['MSFT', 'NVDA', 'AAPL', 'AMZN', 'META', 'AVGO', 'GOOGL', 'TSLA']

    # securities = ['AMD']

    '''
    securities = [
        "MSFT","NVDA","AAPL","AVGO","GOOGL","ORCL","IBM","CSCO","CRM","AMD","INTU","NOW","TXN","QCOM",
        "ADBE","AMAT","PLTR","ACN","META","NFLX","GOOGL","GOOG","DIS","VZ","T","UBER","AMZN","TSLA","HD",
        "MCD","BKNG","COST","WMT","PG","KO","PM","PEP","BRK/B","JPM","V","MA","BAC","WFC","GS","MS","SPGI",
        "SCHW","AXP","BLK","C","PGR","LLY","JNJ","ABBV","UNH","ABT","MRK","ISRG","BSX","TMO","AMGN","RTX",
        "XOM","CVX","GE","CAT","BA","HON","LIN","NEE","DUK","SO","AEP","D","AMT","PLD","EQIX","O","CBRE"]
    '''
    
    _ = pipeline_actual(securities=securities, CONFIG=CONFIG, filename=filename)

if __name__ == "__main__":
    main()




MSFT
combo_total= 2


------------------------------------------------------------------------
(2/2) unrestricted | Params: {'n_regimes': 4, 'dim_latent': 1, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


------------------------------------------------------------------------
(1/2) unrestricted | Params: {'n_regimes': 3, 'dim_latent': 1, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


ELBO: 1663.4: 100%|██████████| 10/10 [00:00<00:00, 28.72it/s]



fits succeeded: 10

LEADERBOARD:
 rank     score  n_regimes  dim_latent  single_subspace
    1 -0.035850          3           1             True
    2 -0.084647          4           1             True
combo_total= 2


------------------------------------------------------------------------
(2/2) unrestricted | Params: {'n_regimes': 4, 'dim_latent': 2, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


------------------------------------------------------------------------
(1/2) unrestricted | Params: {'n_regimes': 3, 'dim_latent': 2, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


ELBO: 2532.9: 100%|██████████| 10/10 [00:00<00:00, 22.29it/s]



fits succeeded: 10

LEADERBOARD:
 rank     score  n_regimes  dim_latent  single_subspace
    1 -0.038255          3           2             True
    2 -0.050590          4           2             True
combo_total= 2


------------------------------------------------------------------------
(1/2) unrestricted | Params: {'n_regimes': 3, 'dim_latent': 2, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


------------------------------------------------------------------------
(2/2) unrestricted | Params: {'n_regimes': 4, 'dim_latent': 2, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


...


fits succeeded: 10

LEADERBOARD:
 rank     score  n_regimes  dim_latent  single_subspace     model_name
    1 -0.004204          3           6             True factor2_ff5mom
    2 -0.007545          4           6             True factor2_ff5mom


NVDA
combo_total= 2


------------------------------------------------------------------------
(2/2) unrestricted | Params: {'n_regimes': 4, 'dim_latent': 1, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


------------------------------------------------------------------------
(1/2) unrestricted | Params: {'n_regimes': 3, 'dim_latent': 1, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


ELBO: 1380.3: 100%|██████████| 10/10 [00:00<00:00, 28.23it/s]



fits succeeded: 10

LEADERBOARD:
 rank     score  n_regimes  dim_latent  single_subspace
    1 -0.019069          4           1             True
    2 -0.031085          3           1             True
combo_total= 2


------------------------------------------------------------------------
(1/2) unrestricted | Params: {'n_regimes': 3, 'dim_latent': 2, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


------------------------------------------------------------------------
(2/2) unrestricted | Params: {'n_regimes': 4, 'dim_latent': 2, 'single_subspace': True};{'train_window': 756, 'overlap_window': 63}
------------------------------------------------------------------------


ELBO: 2198.4:  30%|███       | 3/10 [00:00<00:00, 21.49it/s]


KeyboardInterrupt: 